<a href="https://colab.research.google.com/github/mcnica89/MATH4060/blob/main/Week3B_time_to_hit_in_Gamblers_ruin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import itertools
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax import random as jrandom
from jax import nn as jnn
from jax import jit
import random
import time
import timeit
import math
import numpy as np

plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)
font = {'size'   : 20}

plt.rc('font', **font)

jnp.set_printoptions(precision=3,suppress=True)

In [42]:
def gamblers_ruin_matrix(N_target):
  transition_matrix = np.zeros((N_target+1,N_target+1))
  i,j = np.indices(transition_matrix.shape)
  transition_matrix[i==j-1] = 0.5
  transition_matrix[i==j+1] = 0.5

  transition_matrix[0,0] = 1
  transition_matrix[0,1] = 0
  
  transition_matrix[N_target,N_target] = 1
  transition_matrix[N_target,N_target-1] = 0
  
  return transition_matrix

In [43]:
N_target = 10

In [44]:
P = gamblers_ruin_matrix(N_target)
print(P)

[[1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]]


In [45]:
M = np.eye(11)-P
print(M)

[[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [-0.5  1.  -0.5  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.  -0.5  1.  -0.5  0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -0.5  1.  -0.5  0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  -0.5  1.  -0.5  0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  -0.5  1.  -0.5  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -0.5  1.  -0.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5  0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5  0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]]


In [46]:
v = np.ones(N_target+1)
v[0] = 0
v[N_target] = 0
print(v)

[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]


In [47]:
jnp.linalg.solve(M,v)

DeviceArray([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [48]:
M_modified = M
M_modified[0,0]=1
M_modified[N_target,N_target]=1
print(M_modified)

[[ 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [-0.5  1.  -0.5  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.  -0.5  1.  -0.5  0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -0.5  1.  -0.5  0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  -0.5  1.  -0.5  0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  -0.5  1.  -0.5  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -0.5  1.  -0.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5  0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5  0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.  -0.5  1.  -0.5]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1. ]]


In [26]:
jnp.linalg.solve(M_modified,v)

DeviceArray([ 0.,  9., 16., 21., 24., 25., 24., 21., 16.,  9.,  0.], dtype=float32)

In [57]:
#Estimate using (1-x)^{-1} = 1+x+x^2+x^3...
jnp.set_printoptions(precision=2,suppress=True)
N_terms = 200
ans = np.zeros(N_target+1)
Ppower_v = v #Terms of the from P^n (v), starts at P^0 v = v
for i in range(N_terms):
  ans += Ppower_v
  Ppower_v = np.matmul(P,Ppower_v) #Update P^{n+1} = P (P^n v)
  if i%10 == 0:
    print("After ", i, " terms:\n", ans)


After  0  terms:
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
After  10  terms:
 [ 0.    4.41  7.26  8.99  9.86 10.16  9.86  8.99  7.26  4.41  0.  ]
After  20  terms:
 [ 0.    6.22 10.71 13.73 15.44 16.01 15.44 13.73 10.71  6.22  0.  ]
After  30  terms:
 [ 0.    7.32 12.8  16.6  18.82 19.56 18.82 16.6  12.8   7.32  0.  ]
After  40  terms:
 [ 0.    7.98 14.06 18.33 20.86 21.71 20.86 18.33 14.06  7.98  0.  ]
After  50  terms:
 [ 0.    8.38 14.83 19.39 22.1  23.01 22.1  19.39 14.83  8.38  0.  ]
After  60  terms:
 [ 0.    8.63 15.29 20.02 22.85 23.79 22.85 20.02 15.29  8.63  0.  ]
After  70  terms:
 [ 0.    8.77 15.57 20.41 23.3  24.27 23.3  20.41 15.57  8.77  0.  ]
After  80  terms:
 [ 0.    8.86 15.74 20.64 23.58 24.56 23.58 20.64 15.74  8.86  0.  ]
After  90  terms:
 [ 0.    8.92 15.84 20.78 23.74 24.73 23.74 20.78 15.84  8.92  0.  ]
After  100  terms:
 [ 0.    8.95 15.9  20.87 23.85 24.84 23.85 20.87 15.9   8.95  0.  ]
After  110  terms:
 [ 0.    8.97 15.94 20.92 23.91 24.9  23.91 20.92 15.94  